# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning', 'R. E. Hviding']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
B. Maity  ->  B. Maity  |  ['B. Maity']
J. Li  ->  J. Li  |  ['J. Li']
F. Xu  ->  F. Xu  |  ['F. Xu']
F. Xu  ->  F. Xu  |  ['F. Xu']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
I. Hammond  ->  I. Hammond  |  ['I. Hammond']
M. Flock  ->  M. Flock  |  ['M. Flock']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']


J. Braspenning  ->  J. Braspenning  |  ['J. Braspenning']
Arxiv has 112 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2602.00263


extracting tarball to tmp_2602.00263...

 done.


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2602.00263/sample631.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'TableData/PhysTable' from 'tmp_2602.00263/TableData/PhysTable.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'TableData/PulsarTable' from 'tmp_2602.00263/TableData/PulsarTable.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.

...

Issues with the citations
undefined string in line 3287: mn
Retrieving document from  https://arxiv.org/e-print/2602.02275


extracting tarball to tmp_2602.02275... done.
Retrieving document from  https://arxiv.org/e-print/2602.02484


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2602.02275/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'summaryTable_SD750' from 'tmp_2602.02275/summaryTable_SD750.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'summaryTable_SD1500' from 'tmp_2602.02275/summaryTable_SD1500.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/li

extracting tarball to tmp_2602.02484... done.


Issues with the citations
repeated bibliography entry: Ghirardini_2024


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.00263-b31b1b.svg)](https://arxiv.org/abs/2602.00263) | **The Fourth HAWC Catalog of Very-High-Energy Gamma-Ray Sources**  |
|| R. Alfaro, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2026-02-03*|
|*Comments*| *27 pages, 21 figures, pending submission to The Astrophysical Journal*|
|**Abstract**|            We present an updated catalog of TeV gamma-ray sources based on the fifth pass of data from the High-Altitude Water Cherenkov (HAWC) Observatory. This release benefits from improved event reconstruction and nearly three additional years of observations. It also incorporates a systematic multi-source fitting framework, enabling more flexible and accurate modeling of the gamma-ray sky. This fitting procedure was modeled after the manual approach used in HAWC analyses of individual sources and regions, as well as other gamma-ray catalogs, like the 4FGL. In addition to more varied modeling of source morphology and spectral parameters compared to previous HAWC catalogs, this catalog uses a robust modeling of Galactic diffuse TeV emission. The fitting procedure uses both point-like and symmetric Gaussian spatial templates to model the source morphology. The spectral shape of the emission is modeled with either a simple power-law or log-parabola to explore curvature in the spectral energy distribution. We report 85 sources at the 4{\sigma} level, including 11 sources not associated with any TeVCat source using a distance-based association criterion. Distance-based association with the 1LHAASO catalog results in 22 4HWC sources without a counterpart. Additionally, there are 12 sources not associated with any physical counterpart in the Low- or High-Mass X-Ray Binary, the ATNF, or Fermi Pulsar, or SNR catalogs of sources. Five of the aforementioned sources have no counterpart in any of the catalogs searched and represent an opportunity for follow-up observations.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.00301-b31b1b.svg)](https://arxiv.org/abs/2602.00301) | **Studying dark gaps in Ly-$α$ forest transmission with large reionization simulation**  |
|| <mark>B. Maity</mark>, F. B. Davies, B. Spina, S. E. I. Bosman |
|*Appeared on*| *2026-02-03*|
|*Comments*| *submitted to A&A, comments welcome*|
|**Abstract**|            The physical conditions of the intergalactic medium (IGM) during the final stages of cosmic reionization ($z\sim5.0-6.0$) are not yet fully understood. Recent reports of unexpectedly large-scale ($\ge 150 h^{-1}\mathrm{cMpc}$) correlation in Ly-$\alpha$ transmission flux using extended XQR-30 quasar spectra pose interesting consequences on the reionization end stages. In this work, we investigate the Ly-$\alpha$ forest dark-gap distribution (defined as regions with transmitted flux below 0.05) as another sensitive tracer of the IGM, using an efficient, large-volume ($\sim 1 ~\mathrm{Gpc}$) simulation framework. By constructing a suite of physically motivated model variants (i.e, varying the reionization redshift, IGM temperature, and ionizing-photon mean free path), we generate synthetic sightlines and compare their predicted cumulative distribution of dark gaps with that of observed spectra (at redshift intervals of $\Delta z=0.2$). We find that most of the models achieve qualitatively consistent agreement with the data. Specifically, the scenario involving a slightly later reionization completion ($z\sim 5.4$) provides the closest match, while a short constant mean free path model disfavors the data at lower redshifts. These findings give further support for the emerging scenario of reionization end extending to $z\le5.7$, although they can not rule out a slightly early reionization with enhanced post-ionization ultraviolet (UV) background fluctuations. A similar conclusion arises from the redshift distribution of long dark gap ($L\ge 30 ~h^{-1}\mathrm{cMpc}$) fraction. However, the model variants are still not able to reproduce the observed strong flux correlations at unusually large scales, which remains open for further investigations.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.01238-b31b1b.svg)](https://arxiv.org/abs/2602.01238) | **ATOMS-QUARKS survey: Inflow and infall in massive protocluster G318.049+00.086: Evidence of competitive accretion**  |
|| S. Gupta, et al. -- incl., <mark>F. Xu</mark> |
|*Appeared on*| *2026-02-03*|
|*Comments*| *22 pages, 15 figures, and 3 tables, accepted for publication in ApJ*|
|**Abstract**|            We present a gas kinematic study of the massive protocluster G318.049+00.086. The protocluster is reported to contain 12 prestellar core candidates and 4 protostellar cores. Filamentary structures are identified using the 1.3 mm dust continuum map, with four of them converge into a dense central region, forming a hub-filament system (HFS). High velocity gradients (10 - 20 km s$^{-1}$ pc$^{-1}$) derived from PV analysis of H$^{13}$CO$^{+}$ emission along three of those filaments are suggestive of mass inflow onto the central hub. A mass inflow rate higher than $10^{3}$ M$_{\odot}$ Myr$^{-1}$ along the filaments is indicating that the central hub is capable of forming massive star(s). Investigation of H$^{13}$CO$^{+}$ and CCH spectral profiles revealed the majority of the cores having the characteristic blue asymmetric line profiles, typical signature of gravitational collapse. The remaining few cores showed red asymmetric profiles, indicative of gas expansion. Also, the derived mass infall rates for the protostellar cores in hub-region is significantly higher in comparison to those located along the filaments. The mass-radius relationship of the cores revealed that the cores with red profiles reside in the massive star formation regime. However, the global velocity gradient along the filaments suggests that these particular cores are losing material to the hub. Our results are supporting a competitive accretion scenario of massive star formation where gas is expected to be funnelled from less gravitationally dominant cores to the cores located at the gravitationally favorable position.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.01620-b31b1b.svg)](https://arxiv.org/abs/2602.01620) | **exoALMA XIX: Confirmation of Non-thermal Line Broadening in the DM Tau Protoplanetary Disk**  |
|| C. Hardiman, et al. -- incl., <mark>I. Hammond</mark>, <mark>M. Flock</mark>, <mark>M. Benisty</mark>, <mark>D. Fasano</mark> |
|*Appeared on*| *2026-02-03*|
|*Comments*| *18 pages, 11 figures, published in ApJ Letters*|
|**Abstract**|            Turbulence is expected to transport angular momentum and drive mass accretion in protoplanetary disks. One way to directly measure turbulent motion in disks is through molecular line broadening. DM Tau is one of only a few disks with claimed detection of nonthermal line broadening of 0.25cs-0.33cs, where cs is the sound speed. Using the radiative transfer code mcfost within a Bayesian inference framework that evaluates over five million disk models to efficiently sample the parameter space, we fit high-resolution (0.15", 28 m s-1) 12CO J = 3-2 observations of DM Tau from the exoALMA Large Program. This approach enables us to simultaneously constrain the disk structure and kinematics, revealing a significant nonthermal contribution to the line width of ~0.4cs, inconsistent with purely thermal motions. Using the CO-based disk structure as a starting point, we reproduce the CS J = 7-6 emission well, demonstrating that the CS (which is more sensitive to nonthermal motions than CO) agrees with the turbulence inferred from the CO fit. Establishing a well-constrained background disk model further allows us to identify residual structures in the moment maps that deviate from the expected emission, revealing localized perturbations that may trace forming planets. This framework provides a powerful general approach for extracting disk structure and nonthermal broadening directly from molecular line data and can be applied to other disks with high-quality observations.         |

...

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.00626-b31b1b.svg)](https://arxiv.org/abs/2602.00626) | **Down-bending Breaks in Galactic Disks Are an Intrinsic Byproduct of Inside-out Growth**  |
|| L. Chen, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2026-02-03*|
|*Comments*| *11 pages, 5 figures, submitted to ApJL*|
|**Abstract**|            The exponential profile has long been hypothesized as the fundamental morphology of galactic disks. The IllustrisTNG simulations reproduce diverse surface-density profiles: Type I (single exponential), Type II (down-bending), and Type III (up-bending), consistent with observed mass-size relations and kinematics. Type II disks dominate the stellar-mass regime $M_\star < 10^{10.6} M_\odot$ with a prevalence of about 40%, exhibiting systematically extended morphologies. Conversely, Type III and Type I galaxies are more compact while following the same mass-size scaling relation. Evolutionary histories show that Type II galaxies experience minimal external perturbations, suggesting that Type II disks represent an intrinsic disk form and challenging conventional single-exponential paradigms. We demonstrate that Type II breaks arise naturally via inside-out growth since $z=1$, governed by synchronized cold-gas accretion and localized peaks in specific star formation rate. This mechanism also produces the characteristic U-shaped age profiles of Type II disks. Stellar dynamical redistribution plays a minor role in their formation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.00662-b31b1b.svg)](https://arxiv.org/abs/2602.00662) | **Gamma-Ray Bursts: Evidence for a Common Origin of X-ray Plateaus with Diverse Temporal Decay Index**  |
|| X.-F. Dong, et al. -- incl., <mark>F. Xu</mark> |
|*Appeared on*| *2026-02-03*|
|*Comments*| *14 pages, 8 figures, 2 tables*|
|**Abstract**|            A significant fraction of gamma-ray bursts (GRBs) exhibit a plateau in the early X-ray afterglow light curve, whose mechanism remains uncertain. While the post-plateau normal decay index ($\alpha_2$) is commonly used to constrain the afterglow dynamics, the shallow-decay slope of the plateau itself ($\alpha_1$) has received comparatively little attention. Recent observations, however, reveal substantial dispersion in $\alpha_1$, raising the question of whether GRBs with rising, flat and mildly decaying plateaus represent intrinsically distinct populations. To address this question, we collect a uniform sample of 185 $\textit{Swift}$ GRBs with a well-defined plateau and divide them into three groups based on $\alpha_1$. Using a non-parametric approach, we reconstruct their X-ray luminosity functions, redshift distributions and event rates. It is found that the three groups exhibit statistically consistent properties across all diagnostics, with no evidence for group-specific features. Monte Carlo perturbation tests further show that these results are insensitive to the adopted classification boundaries of $\alpha_1$. Our results indicate that variations in the plateau slope $\alpha_1$ do not define distinct GRB subclasses, but instead the sample constitutes a statistically uniform population governed by a common framework.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.01319-b31b1b.svg)](https://arxiv.org/abs/2602.01319) | **Factorized neural posterior estimation for rapid and reliable inference of parameterized post-Einsteinian deviation parameters in gravitational waves**  |
|| Y.-X. Zhang, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2026-02-03*|
|*Comments*| *15 pages, 6 figures*|
|**Abstract**|            The direct detection of gravitational waves (GWs) by LIGO has strikingly confirmed general relativity (GR), but testing GR via GWs requires estimating parameterized post-Einsteinian (ppE) deviation parameters in waveform models. Traditional Bayesian inference methods like Markov chain Monte Carlo (MCMC) provide reliable estimates but suffer from prohibitive computational costs, failing to meet the real-time demands and surging data volume of future GW detectors. Here, we propose a factorized neural posterior estimation framework: we construct independent normalizing flow models for each of the nine ppE deviation parameters and effectively integrate prior information from other source parameters via a conditional embedding network. Leveraging a hybrid neural network with a convolutional neural network and a Residual Neural Network for feature extraction, our method performs rapid and statistically reliable posterior inference directly from binary black hole signals. Compared to conventional MCMC, our approach achieves millisecond-scale inference time with a speedup factor of $9 \times 10^4$. Comprehensive validations show that the posterior estimates pass the Kolmogorov-Smirnov test and achieve empirical coverage probabilities close to theoretical targets. This work demonstrates the great potential of deep learning for GW parameter estimation and provides a viable technical solution for real-time GR tests with next-generation detectors.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.02275-b31b1b.svg)](https://arxiv.org/abs/2602.02275) | **Search for ultra-high energy neutrons from Galactic sources with the Pierre Auger Observatory**  |
|| P. A. Collaboration, et al. |
|*Appeared on*| *2026-02-03*|
|*Comments*| *13 pages, 6 tables, 1 figure*|
|**Abstract**|            Deflections in the propagation of charged ultra-high-energy cosmic rays (UHECRs) caused by magnetic fields make the identification of their sources challenging. On the other hand, the arrival directions at Earth of neutrons point directly to their origin. The emission of UHECRs from a source is expected to be accompanied by the production of neutrons in its vicinity through interactions with ambient matter and radiation. Since free neutrons travel a mean distance $d/\text{kpc}=9.2(E/\text{EeV})$ before decaying, a neutron flux in the EeV range could be detected on Earth from sources of UHECRs in our Galaxy. Using cosmic-ray data from the Phase\,I of the Surface Detector of the Pierre Auger Observatory, we search for neutron fluxes from Galactic candidate sources. We select more than 1000 objects of astrophysical interest, stacking them into target sets. The targets all have declinations within the exposure of the Observatory, ranging from $-90^\circ$ up to $+45^\circ$ for energies above 1 EeV (and up to $+20^\circ$ for energies down to 0.1 EeV). Given that a neutron air shower is indistinguishable from a proton one, there is a significant background due to cosmic rays. A neutron flux from the direction of a candidate source would be identified by a celestial density of events that significantly exceeds the expected density of cosmic rays for that direction. No significant excess is found at any tested target direction, and an upper limit on the neutron flux is calculated for each candidate source.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2602.00263/./offplane7.png', 'tmp_2602.00263/./000.png', 'tmp_2602.00263/./offplane1.png']
copying  tmp_2602.00263/./offplane7.png to _build/html/
copying  tmp_2602.00263/./000.png to _build/html/
copying  tmp_2602.00263/./offplane1.png to _build/html/
exported in  _build/html/2602.00263.md
    + _build/html/tmp_2602.00263/./offplane7.png
    + _build/html/tmp_2602.00263/./000.png
    + _build/html/tmp_2602.00263/./offplane1.png
found figures ['tmp_2602.00301/./updated_plots/Bigbox_model_snaps.png', 'tmp_2602.00301/./updated_plots/Bigbox_model_snaps_rest.png', 'tmp_2602.00301/./updated_plots/reion_models_bigbox.png']
copying  tmp_2602.00301/./updated_plots/Bigbox_model_snaps.png to _build/html/
copying  tmp_2602.00301/./updated_plots/Bigbox_model_snaps_rest.png to _build/html/
copying  tmp_2602.00301/./updated_plots/reion_models_bigbox.png to _build/html/
exported in  _build/html/2602.00301.md
    + _build/html/tmp_2602.00301/./updated_plots/Bigbox_model_snaps.png
 

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\Like}{\mathcal{L}}$
$\newcommand{\LogLike}[1]{\log{\left(\Like_{#1}\right)}}$</div>



<div id="title">

# The Fourth HAWC Catalog of Very-High-Energy Gamma-Ray Sources

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.00263-b31b1b.svg)](https://arxiv.org/abs/2602.00263)<mark>Appeared on: 2026-02-03</mark> -  _27 pages, 21 figures, pending submission to The Astrophysical Journal_

</div>
<div id="authors">

R.~Alfaro, et al.

</div>
<div id="abstract">

**Abstract:** We present an updated catalog of TeV gamma-ray sources based on the fifth pass of data from the High-Altitude Water Cherenkov (HAWC) Observatory. This release benefits from improved event reconstruction and nearly three additional years of observations. It also incorporates a systematic multi-source fitting framework, enabling more flexible and accurate modeling of the gamma-ray sky. This fitting procedure was modeled after the manual approach used in HAWC analyses of individual sources and regions as well as other gamma-ray catalogs, like the 4FGL. In addition to more varied modeling of source morphology and spectral parameters compared to previous HAWC catalogs, this catalog uses a robust modeling of Galactic diffuse TeV emission. The fitting procedure uses both point-like and symmetric Gaussian spatial templates to model the source morphology. The spectral shape of the emission is modeled with either a simple power-law or log-parabola to explore curvature in the spectral energy distribution. We report 85 sources at the 4 $\sigma$ level, including 11 sources not associated with any TeVCat source using a distance-based association criterion. Distance-based association with the 1LHAASO catalog results in 22 4HWC sources without a counterpart. Additionally, there are 12 sources not associated with any physical counterpart in the Low- or High-Mass X-Ray Binary, the ATNF, or _Fermi_ Pulsar, or SNR catalogs of sources. Five of the aforementioned sources have no counterpart in any of the catalogs searched and represent an opportunity for follow-up observations.

</div>

<div id="div_fig1">

<img src="tmp_2602.00263/./offplane7.png" alt="Fig6" width="100%"/>

**Figure 6. -** HAWC significance map of region R.A. = 253.52◦ee \space and Dec. = 39.74◦ee \space with the 4HWC source overlaid as a dashed circle with radius equal to the extension parameter. A small dot represents the center of the source model emission. Map generated using a point-source assumption with a $-2.6$ power-law index. (*fig:Offplane7*)

</div>
<div id="div_fig2">

<img src="tmp_2602.00263/./000.png" alt="Fig13" width="100%"/>

**Figure 13. -** HAWC significance map of region $l$ = $-20$◦ee \space to $l$ = 20◦ee, $\lvert b \rvert <$ 10◦ee \space with the 4HWC sources overlaid as dashed circles with radius equal to the extension parameter. A small dot represents the center of the source model emission. Map generated using a point-source assumption with a $-2.6$ power-law index. The white or transparent portion of the map represents empty portions of the sky beyond the zenith cut of the data set. (*fig:Galplot1*)

</div>
<div id="div_fig3">

<img src="tmp_2602.00263/./offplane1.png" alt="Fig1" width="100%"/>

**Figure 1. -** HAWC significance map of region R.A. = 95.54◦ee \space and Dec. = 37.78◦ee \space with the 4HWC source overlaid as a dashed circle with radius equal to the extension parameter. A small dot represents the center of the source model emission. Map generated using a point-source assumption with a $-2.6$ power-law index. (*fig:Offplane1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.00263"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\be}{\begin{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand\bear{#1}\ear{\begin{align}#1\end{align}}$
$\newcommand{\nline}{\notag \\}$
$\newcommand{\f}{\frac}$
$\newcommand{\de}{\mathrm{d}}$
$\newcommand{\del}{\partial}$
$\newcommand{\half}{\frac{1}{2}}$
$\newcommand{\im}{\mathrm{i}}$
$\newcommand{\e}{\mathrm{e}}$
$\newcommand{\Msun}{\mathrm{M}_{\odot}}$
$\newcommand{\eqn}[1]{equation~(\ref{#1})}$
$\newcommand{\eqns}[2]{equations~(\ref{#1}) and~(\ref{#2})}$
$\newcommand{\secn}[1]{Section~\ref{#1}}$
$\newcommand{\appndx}[1]{Appendix~\ref{#1}}$
$\newcommand{\fig}[1]{Fig.~\ref{#1}}$
$\newcommand{\figs}[1]{Figs.~\ref{#1}}$
$\newcommand{\tab}[1]{Table~\ref{#1}}$
$\newcommand{\BM}[1]{{\color{blue}[{\bf }#1]}}$
$\newcommand{\red}[1]{{\color{red} #1}}$
$\usepackage[T1]{fontenc}$
$\usepackage{amsmath}$
$\usepackage{orcidlink}$
$\DeclareRobustCommand{\VAN}[3]{#2}$
$\let\VANthebibliography\thebibliography$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\usepackage{graphicx}$
$\usepackage{txfonts}$
$\begin{document}$
$   \title{Studying dark gaps in Ly-\alpha forest transmission with large reionization simulation}$
$    \authorrunning{Maity et al.}$
$   \author{Barun Maity\inst{1}\fnmsep\thanks{maity@mpia.de}\orcidlink{0000-0002-4682-6970}$
$          ,$
$          Frederick B. Davies$
$          \inst{1}\orcidlink{0000-0003-0821-3644},$
$          Benedetta Spina \inst{2}\orcidlink{0000-0003-1634-1283} \and$
$          Sarah E. I. Bosman \inst{2}\orcidlink{0000-0001-8582-7012}}$
$   \institute{Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany$
$   \and Institute for Theoretical Physics, Heidelberg University, Philosophenweg 12, D–69120, Heidelberg, Germany}$
$   \date{Received XXX; accepted XXX}$
$  \abstract{The physical conditions of the intergalactic medium (IGM) during the final stages of cosmic reionization (z\sim5.0-6.0) are not yet fully understood. Recent reports of unexpectedly large-scale (\ge 150 h^{-1}\mathrm{cMpc}) correlation in Ly-\alpha transmission flux using extended XQR-30 quasar spectra pose interesting consequences on the reionization end stages. In this work, we investigate the Ly-\alpha forest dark-gap distribution (newcommandined as regions with transmitted flux below 0.05) as another sensitive tracer of the IGM, using an efficient, large-volume (\sim 1 ~\mathrm{Gpc}) simulation framework. By constructing a suite of physically motivated model variants (i.e, varying the reionization redshift, IGM temperature, and ionizing-photon mean free path), we generate synthetic sightlines and compare their predicted cumulative distribution of dark gaps with that of observed spectra (at redshift intervals of \Delta z=0.2). We find that most of the models achieve qualitatively consistent agreement with the data. Specifically, the scenario involving a slightly later reionization completion (z\sim 5.4) news the closest match, while a short constant mean free path model disfavors the data at lower redshifts. These findings give further support for the emerging scenario of reionization end extending to z\le5.7, although they can not rule out a slightly early reionization with enhanced post-ionization ultraviolet (UV) background fluctuations. A similar conclusion arises from the redshift distribution of long dark gap (L\ge 30 ~h^{-1}\mathrm{cMpc}) fraction. However, the model variants are still not able to reproduce the observed strong flux correlations at unusually large scales, which remains open for further investigations.$
$}$
$   \keywords{intergalactic medium -- cosmology: theory – dark ages, reionization, first stars -- large-scale structure of Universe}$
$   \maketitle$
$\n\end{document}\end{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand{\nline}{\notag \\}$
$\newcommand{\f}{\frac}$
$\newcommand{\de}{\mathrm{d}}$
$\newcommand{\del}{\partial}$
$\newcommand{\half}{\frac{1}{2}}$
$\newcommand{\im}{\mathrm{i}}$
$\newcommand{\e}{\mathrm{e}}$
$\newcommand{\Msun}{\mathrm{M}_{\odot}}$
$\newcommand{\eqn}[1]{equation~(\ref{#1})}$
$\newcommand{\eqns}[2]{equations~(\ref{#1}) and~(\ref{#2})}$
$\newcommand{\secn}[1]{Section~\ref{#1}}$
$\newcommand{\appndx}[1]{Appendix~\ref{#1}}$
$\newcommand{\fig}[1]{Fig.~\ref{#1}}$
$\newcommand{\figs}[1]{Figs.~\ref{#1}}$
$\newcommand{\tab}[1]{Table~\ref{#1}}$
$\newcommand{\BM}[1]{{\color{blue}[{\bf }#1]}}$
$\newcommand{\red}[1]{{\color{red} #1}}$
$\newcommand\bear{#1}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Studying dark gaps in Ly-$\alpha$ forest transmission with large reionization simulation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.00301-b31b1b.svg)](https://arxiv.org/abs/2602.00301)<mark>Appeared on: 2026-02-03</mark> -  _submitted to A&A, comments welcome_

</div>
<div id="authors">

<mark>B. Maity</mark>, F. B. Davies, B. Spina, S. E. I. Bosman

</div>
<div id="abstract">

**Abstract:** The physical conditions of the intergalactic medium (IGM) during the final stages of cosmic reionization ( $z\sim5.0-6.0$ ) are not yet fully understood. Recent reports of unexpectedly large-scale ( $\ge 150 h^{-1}\mathrm{cMpc}$ ) correlation in Ly- $\alpha$ transmission flux using extended XQR-30 quasar spectra pose interesting consequences on the reionization end stages. In this work, we investigate the Ly- $\alpha$ forest dark-gap distribution (defined as regions with transmitted flux below 0.05) as another sensitive tracer of the IGM, using an efficient, large-volume ( $\sim 1  \mathrm{Gpc}$ ) simulation framework. By constructing a suite of physically motivated model variants (i.e, varying the reionization redshift, IGM temperature, and ionizing-photon mean free path), we generate synthetic sightlines and compare their predicted cumulative distribution of dark gaps with that of observed spectra (at redshift intervals of $\Delta z=0.2$ ). We find that most of the models achieve qualitatively consistent agreement with the data. Specifically, the scenario involving a slightly later reionization completion ( $z\sim 5.4$ ) provides the closest match, while a short constant mean free path model disfavors the data at lower redshifts. These findings give further support for the emerging scenario of reionization end extending to $z\le5.7$ , although they can not rule out a slightly early reionization with enhanced post-ionization ultraviolet (UV) background fluctuations. A similar conclusion arises from the redshift distribution of long dark gap ( $L\ge 30  h^{-1}\mathrm{cMpc}$ ) fraction. However, the model variants are still not able to reproduce the observed strong flux correlations at unusually large scales, which remains open for further investigations.

</div>

<div id="div_fig1">

<img src="tmp_2602.00301/./updated_plots/Bigbox_model_snaps.png" alt="Fig3" width="100%"/>

**Figure 3. -** Lightcone snapshots for three different cases in (_top_: fiducial, _middle_: cons mfp / wo neutral, and _bottom_: low temp) in three rows. The columns correspond to density ($\Delta$), neutral fractions ($x_{\mathrm{HI}}$), UVB fluctuations ($\Gamma_{\mathrm{HI}}/\langle \Gamma_{\mathrm{HI}}\rangle$), temperature ($T$), and flux ($F$). The colorbars have been shown in logarithmic scales. The rest of the scenarios has been shown in Appendix \ref{app:appendix1}. (*fig:mdel_snap*)

</div>
<div id="div_fig2">

<img src="tmp_2602.00301/./updated_plots/Bigbox_model_snaps_rest.png" alt="Fig1" width="100%"/>

**Figure 1. -** Lightcone snapshots for the rest of the five cases complementing Figure \ref{fig:mdel_snap}(i.e., "late", "early",  "small mfp", "large mfp" and "high temp", respectively from top to bottom). The columns correspond to density ($\Delta$), neutral fractions ($x_{\mathrm{HI}}$), UVB fluctuations ($\Gamma_{\mathrm{HI}}/\langle \Gamma_{\mathrm{HI}}\rangle$), temperature ($T$), and flux ($F$). The colorbars have been shown in logarithmic scales. (*fig:mdel_snap_appndix*)

</div>
<div id="div_fig3">

<img src="tmp_2602.00301/./updated_plots/reion_models_bigbox.png" alt="Fig2" width="100%"/>

**Figure 2. -** Different observables for the various model scenarios assumed in this study. From left to right, the panels show redshift evolution of global neutral fraction ($Q_{\mathrm{HI}}^V$), mean IGM temperature ($T_0$), index of temperature-density relation ($\gamma$) and the effective photon mean free path ($\lambda_0$). We also show various constraints on these quantities, as suggested by recent studies, i.e, constraints on neutral fraction \citep[][]{Davies18, 2022MNRAS.512.5390G,2023ApJ...942...59J, 2024A&A...688L..26S, Zhu2024_damping}, IGM temperature estimates \citep[][]{2020MNRAS.494.5091G}, mean free path estimates \citep[][]{2021MNRAS.508.1853B,Gaikwad23,2023ApJ...955..115Z, Davies24}. (*fig:model_var*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.00301"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\thefigure}{A.\arabic{figure}}$
$\newcommand{\thetable}{A.\arabic{table}}$</div>



<div id="title">

# ATOMS-QUARKS survey: Inflow and infall in massive protocluster G318.049+00.086: Evidence of competitive accretion

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.01238-b31b1b.svg)](https://arxiv.org/abs/2602.01238)<mark>Appeared on: 2026-02-03</mark> -  _22 pages, 15 figures, and 3 tables, accepted for publication in ApJ_

</div>
<div id="authors">

S. Gupta, et al. -- incl., <mark>F. Xu</mark>

</div>
<div id="abstract">

**Abstract:** We present a gas kinematic study of the massive protocluster G318.049+00.086. The protocluster is reported to contain 12 prestellar core candidates and 4 protostellar cores. Filamentary structures are identified using the 1.3 mm dust continuum map, with four of them converge into a dense central region, forming a hub-filament system (HFS). High velocity gradients (10 - 20 km s $^{-1}$ pc $^{-1}$ ) derived from PV analysis of H $^{13}$ CO $^{+}$ emission along three of those filaments are suggestive of mass inflow onto the central hub. A mass inflow rate higher than $10^{3}$ M $_{\odot}$ Myr $^{-1}$ along the filaments is indicating that the central hub is capable of forming massive star(s). Investigation of H $^{13}$ CO $^{+}$ and CCH spectral profiles revealed the majority of the cores having the characteristic blue asymmetric line profiles, typical signature of gravitational collapse. The remaining few cores showed red asymmetric profiles, indicative of gas expansion. Also, the derived mass infall rates for the protostellar cores in hub-region is significantly higher in comparison to those located along the filaments. The mass-radius relationship of the cores revealed that the cores with red profiles reside in the massive star formation regime. However, the global velocity gradient along the filaments suggests that these particular cores are losing material to the hub. Our results are supporting a competitive accretion scenario of massive star formation where gas is expected to be funnelled from less gravitationally dominant cores to the cores located at the gravitationally favorable position.

</div>

<div id="div_fig1">

<img src="tmp_2602.01238/./vel_grad_12_34_H13COp_errorbars.png" alt="Fig4" width="100%"/>

**Figure 4. -** The upper panel shows the PV diagram along the filament path F1 and F2, and the lower panel shows the PV along F3 and F4. The PV diagrams are uniformly sampled along the filament skeletons with a step size of 0.016 pc, corresponding to half of the synthesized beam size at the source distance. The velocity error bars represent the standard deviation of the velocities within each sampled bin along the offset. The vertical-dashed line in each panel marks the position of the hub. The orange, purple and red lines indicate the least squares fit to F1, F3 and F4 segments of the PV diagrams in both the panels, respectively. (*fig:vel_grad*)

</div>
<div id="div_fig2">

<img src="tmp_2602.01238/./filament41234_cropped.png" alt="Fig1" width="100%"/>

**Figure 1. -** (a) The identified H$^{13}$CO$^{+}$ filaments overlaid on the moment 0 map of H$^{13}$CO$^{+}$ above the $5\sigma$ level. The red colored filament marks the longest filament and the green shows other filaments present in the region. (b) Displays the filaments (orange lines) identified in the 1.3 mm continuum emission overlaid on the 1.3 mm dust continuum map of the protocluster. The primary filaments are also labeled as F1, F2, F3, and F4. (c) Overlay of H$^{13}$CO$^{+}$  and 1.3 mm  filaments on H$^{13}$CO$^{+}$ moment 0. In all the panels, the identified cores by \citet{2024ApJS..270....9X} are shown and labeled. (*fig:filament*)

</div>
<div id="div_fig3">

<img src="tmp_2602.01238/./filament_velocity_crop.png" alt="Fig3" width="100%"/>

**Figure 3. -** Filaments identified in the 1.3 mm continuum emission overlaid on the moment 1 map of H$^{13}$CO$^{+}$(1--0). The olive and green colors mark the primary filament paths (i.e., F1-F2 and F3-F4, respectively) used to derive the PV diagrams. (*fig:moment1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.01238"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\Msun}{\ensuremath{{M}_{\odot}}}$
$\newcommand{\Lsun}{\ensuremath{{\rm L_{\odot}}}}$
$\newcommand{\Rsun}{\ensuremath{{\rm R_{\odot}}}}$
$\newcommand{\Mjup}{{\rm M}_{\rm Jup}}$
$\newcommand{\ih}[1]{\textcolor{blue}{\textbf{IH:} #1}}$
$\newcommand{\ch}[1]{\textcolor{purple}{\textbf{CH:} #1}}$
$\newcommand{\cass}[1]{\textcolor{teal}{\textbf{Cass:} #1}}$
$\newcommand{\td}[1]{\textcolor{cyan}{\textbf{TD:} #1}}$
$\newcommand{\tom}[1]{\textcolor{magenta}{\textbf{TH:} #1}}$
$\newcommand{\cp}[1]{\textcolor{red}{\textbf{CP:} #1}}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$</div>



<div id="title">

# exoALMA XIX: Confirmation of non-thermal line broadening in the DM Tau protoplanetary disk

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.01620-b31b1b.svg)](https://arxiv.org/abs/2602.01620)<mark>Appeared on: 2026-02-03</mark> -  _18 pages, 11 figures, published in ApJ Letters_

</div>
<div id="authors">

C. Hardiman, et al. -- incl., <mark>I. Hammond</mark>, <mark>M. Flock</mark>, <mark>M. Benisty</mark>, <mark>D. Fasano</mark>

</div>
<div id="abstract">

**Abstract:** Turbulence is expected to transport angular momentum and drive mass accretion in protoplanetary disks. One way to directly measure turbulent motion in disks is through molecular line broadening.DM Tau is one of only a few disks with claimed detection of non-thermal line broadening of 0.25--0.33 $c_{\rm s}$ , where $c_{\rm s}$ is the sound speed.Using the radiative transfer code ${\sc mcfost}$ within a Bayesian inference framework that evaluates over five million disk models to efficiently sample the parameter space, we fit high-resolution ( $0\farcs15$ , 28 m s $^{-1}$ ) $^{12}$ CO $J = 3 - 2$ observations of DM Tau from the exoALMA Large Program.This approach enables us to simultaneously constrain the disk structure and kinematics, revealing a significant non-thermal contribution to the line width of $\sim$ 0.4 $c_{\rm s}$ , inconsistent with purely thermal motions.Using the CO-based disk structure as a starting point, we reproduce the CS $J = 7 - 6$ emission well, demonstrating that the CS (which is more sensitive to non-thermal motions than CO) agrees with the turbulence inferred from the CO fit.Establishing a well-constrained background disk model further allows us to identify residual structures in the moment maps that deviate from the expected emission, revealing localized perturbations that may trace forming planets.This framework provides a powerful general approach for extracting disk structure and non-thermal broadening directly from molecular line data, and can be applied to other disks with high-quality observations.

</div>

<div id="div_fig1">

<img src="tmp_2602.01620/./dm_tau_co_lineprofile_2.png" alt="Fig3.1" width="50%"/><img src="tmp_2602.01620/./dm_tau_cs_lineprofile_2.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** Comparison of observed and modeled integrated line profiles for DM Tau. _Left_: CO $J$=3–2 line profile. _Right_: CS $J$=7–6 line profile. The vertical dotted lines on each panel mark the velocity channels included in the fit. The agreement for CS demonstrates that the CO-derived disk structure can be directly applied to other molecular tracers, allowing differences in emission to be interpreted primarily in terms of chemistry and excitation rather than large-scale structural variations. (*fig: lineprofiles*)

</div>
<div id="div_fig2">

<img src="tmp_2602.01620/./turb_vs_noturb_smaller.png" alt="Fig2" width="100%"/>

**Figure 2. -** Channel maps of DM Tau in $^{12}$CO $J$=3–2 at 0$\farcs$15 resolution. The top row shows the observed data, the second row our best-fit model with non-zero turbulence, and the third row the residuals from subtracting this model from the data. The fourth and fifth rows show the model and residuals for the best fit zero-turbulence model for comparison. Each residual color bar spans $\pm3\sigma$ flux for the data cube. (*fig:image best CO*)

</div>
<div id="div_fig3">

<img src="tmp_2602.01620/./co_number_density_comparison.jpeg" alt="Fig4" width="100%"/>

**Figure 4. -** CO number density in DM Tau. The cyan and magenta dashed lines show the exoALMA-derived emission heights from \citet{Galloway-Sprietsma_exoALMA} using DiscMiner and \texttt{disksurf}. _Left_: $J=2-1$ model from \citet{flaherty20}. _Right_: Our $J=3-2$ model. (*fig:emitting layers co*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.01620"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The FLAMINGO Project: Exploring the X-ray--cosmic-shear cross-correlation as a probe of large-scale structure

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.02484-b31b1b.svg)](https://arxiv.org/abs/2602.02484)<mark>Appeared on: 2026-02-03</mark> - 

</div>
<div id="authors">

W. McDonald, et al. -- incl., <mark>J. Braspenning</mark>

</div>
<div id="abstract">

**Abstract:** Baryonic feedback processes associated with galaxy formation directly influence the large-scale structure by redistributing gas. Recent measurements of the kinetic Sunyaev-Zel’dovich effect and stacks of X-ray emission from optically selected galaxy clusters suggest that feedback from Active Galactic Nuclei (AGN) is more efficient at expelling gas from low-mass clusters than previously thought. The measurement of the cross-correlation between cosmic shear and diffuse X-ray emission provides a new probe of the distribution of gas in groups and clusters. We use the FLAMINGO cosmological, hydrodynamical simulations to examine the X-ray--cosmic-shear cross-correlation. The cross-correlation is most sensitive to the distribution of gas in haloes with masses $10^{14}\leq M_{200\mathrm{c}}/\mathrm{M}_{\odot}\leq10^{15}$ . It is sensitive to the strength of feedback, but the effects of variations in cosmology and baryonic physics are largely degenerate. We compare the FLAMINGO predictions with the cross-correlation between cosmic shear from the Dark Energy Survey and ROSAT all-sky X-ray maps.  We find that, if we neglect the X-ray emission from AGN that would remain unresolved by ROSAT, then the fiducial FLAMINGO model is in excellent agreement with the data, while models with stronger or weaker feedback are ruled out. However, if we account for unresolved AGN, either using the direct FLAMINGO predictions or by abundance matching to the observed (extrapolated) AGN luminosity function, then models with stronger feedback are preferred. We conclude that to exploit the potential of the X-ray--lensing cross-correlation, it will be necessary to resolve fainter AGN, and to use external constraints to break the degeneracy between baryonic feedback and cosmology.

</div>

<div id="div_fig1">

<img src="tmp_2602.02484/./Figures_latest/XK_pipeline2/XK_convolved_flamingo_diff_agn_example_bin_fgas_only.png" alt="Fig4" width="100%"/>

**Figure 4. -** 
    The predicted ROSAT--DES-Y3 X-ray–lensing cross-correlation, in the third tomographic bin (x-3), for the gas fraction variations (fgas$\pm N\sigma$) and fiducial L1$\_$m9 simulation (solid curves) compared to the measurements of \citet[][]{Ferreira_2024}(black points and error bars). All X-ray emission is convolved with the ROSAT response matrix. _Top_: The predicted cross-correlation when only the X-ray emission from hot gas is included._Centre_: We include the X-ray emission from both hot gas and unresolved AGN given by the base BHs. _Bottom_: The same as the centre panel, but the AGN contamination is given by the AM BHs. The base BHs and AM BHs are described in \S\ref{sec:BH_luminosities_and_selection}. The dotted curves give the cross-correlation of the X-ray emission from each set of unresolved AGN and DES-Y3 all-sky maps ($C_{\ell}^{\mathrm{AGN} \kappa}$) for the simulation of the corresponding colour. The $\chi^2$ value of the cross-correlation, in the third tomographic bin, for the fiducial L1$\_$m9 simulation (dark green) and the observational data is printed on the right hand side of each panel. The values in parentheses are the $\Delta \chi^2$(see text) for each cluster gas fraction (indicated by colour).The inclusion of unresolved AGN increases the power of the cross-correlation and more so for the AM BHs.  (*figure:convolved_native_agn_example*)

</div>
<div id="div_fig2">

<img src="tmp_2602.02484/./Figures_latest/XK_pipeline2/XK_normalised_diffuse_gas_example_bin.png" alt="Fig21" width="100%"/>

**Figure 21. -** 
    The cosmology and baryonic feedback dependence of the predicted X-ray--lensing cross-correlation.
    In each main panel, the coloured curves correspond to the predicted cross-correlations in the third tomographic bin (x-3) for the different L1$\_$m9  simulations as indicated by the legend (see \S\ref{sec:flamingo_overview} and Table \ref{tab:cosmo_table}). The lower sub-panels compare the variations
    to the fiducial L1$\_$m9 (dark green) simulation (i.e. $C_{\ell}^{\mathrm{x} \kappa }/\mathrm{L1}\_\mathrm{m9}$).
    Note we only consider the X-ray emission from hot gas and do not convolve the X-ray emission with the response matrix of any telescope.
    _Left_: The dependence on the cluster gas fraction ($\mathrm{fgas}$) that the model has been calibrated to.
    Reducing the cluster gas fraction (mainly by increasing the strength of AGN feedback) suppresses the power of the cross-correlation over all angular scales considered.
    _Centre_: The predicted cross-correlations for the remaining baryonic feedback variations.
    Reducing the stellar mass function suppresses the signal on all scales, however this effect is secondary to varying the cluster gas fraction.
    The choice of AGN feedback model (jet or thermal mode of feedback) has a very minor scale-dependent effect.
    _Right_: The dependence on cosmology and neutrino mass.
    The cross-correlation for the LS8 cosmology is suppressed relative to the fiducial model over all scales shown. Reducing the cluster gas fraction for the LS8 cosmology (LS8$\_\mathrm{fgas}-8\sigma$) further suppresses the signal.
    Increasing the summed neutrino mass relative to the fiducial value ($0.06$ eV) reduces the power over all scales shown.
    Comparing between the three panels, it is clear that the effects of varying the cosmology and baryonic feedback are largely degenerate.
     (*figure:normalised_diffuse_gas_example*)

</div>
<div id="div_fig3">

<img src="tmp_2602.02484/./Figures_latest/XK_pipeline2/XK_L1_m9_large_example.png" alt="Fig20" width="100%"/>

**Figure 20. -** The predicted ROSAT--DES-Y3 X-ray--lensing cross-correlation from the L1$\_$m9 fiducial FLAMINGO simulation (green curves) compared to the observational data points of \citet[][]{Ferreira_2024}(black points and error bars). The numbers in the upper left corner of each panel indicate the DES-Y3 tomographic bin of the shown cross-correlations (e.g. x-1, indicates the cross-correlation of the X-ray signal with the first tomographic bin of the lensing data). Here, the X-ray emission of the L1$\_$m9 simulation is convolved with the ROSAT response matrix (see \S\ref{sec:Xray_diffuse_gas_lightcone_methods}). The solid curves correspond to the predicted cross-correlation if only the X-ray emission from hot gas is included, i.e. it is contamination free. The dotted curves include the X-ray emission from unresolved AGN given by the base FLAMINGO BHs. The dashed curves (AM) depict the AGN contaminated cross-correlations when the unresolved AGN are given by the AM BHs. The shaded regions ($\Delta{\mathrm{AM}}$) indicate the full range of possible cross-correlations from minimising and maximising the AGN contamination given by the $\Delta{\mathrm{AM}}$ BHs as described in the text. All sets of BHs (base, AM and $\Delta\mathrm{AM}$) are described in \S\ref{sec:BH_luminosities_and_selection}). The inset in the top right panel depicts the normalised DES-Y3 source distribution, given by figure 2. of \citet[][]{Doux_2022}, and each tomographic bin (1 through 4, moving left to right) is individually coloured. Without the inclusion of unresolved AGN the simulation reproduces the measurements. However, AGN contamination increases the power of the cross-correlation over all angular scales considered, reducing the agreement with the data. (*figure:plot_1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.02484"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

124  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

20  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
